# Quantum Volume Optimizer

## A World-Class Deep Agent Example

This notebook demonstrates a sophisticated multi-agent system that finds the optimal **Quantum Volume (QV)** configuration for any IBM Quantum backend. It showcases the power of combining all Qiskit MCP servers with LangChain's Deep Agents framework.

### What is Quantum Volume?

Quantum Volume is a single-number metric that captures the largest random circuit of equal width and depth that a quantum computer can successfully implement. A QV of 2^n means the device can reliably execute n-qubit circuits of depth n.

Key factors affecting QV:
- **Two-qubit gate fidelity** (most important)
- **Qubit connectivity** (linear chains need SWAP gates)
- **Coherence times** (T1, T2)
- **Readout accuracy**

## Architecture

```
                    QUANTUM VOLUME OPTIMIZER
                      (Coordinator Agent)
                             |
          +------------------+------------------+------------------+
          |                  |                  |                  |
          v                  v                  v                  v
   BACKEND ANALYST    QUBIT CHAIN        TRANSPILER         QV EXPERIMENT
                      OPTIMIZER          BENCHMARKER        RUNNER
          |                  |                  |                  |
          v                  v                  v                  v
   qiskit-ibm-        qiskit-ibm-       qiskit-ibm-        qiskit-ibm-
   runtime-mcp        runtime-mcp       transpiler-mcp     runtime-mcp
   (backends)         (QV qubit tools)  + qiskit-mcp       (run_sampler,
                                                           get_job_results)
```

The **Qubit Chain Optimizer** uses algorithmic tools:
- `find_optimal_qv_qubits_tool`: Finds densely connected subgraphs for QV
- `find_optimal_qubit_chains_tool`: Finds optimal linear chains
- `get_coupling_map_tool`: Gets backend connectivity graph

**Note**: QV circuit generation with heavy outputs, HOP calculation, and statistical
analysis are performed locally using helper functions in this notebook.

## Setup

```bash
pip install deepagents langchain langchain-mcp-adapters python-dotenv
pip install langchain-anthropic
pip install qiskit-mcp-servers
```

In [ ]:
import os

from deepagents import create_deep_agent
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient


load_dotenv()

print("Configuration:")
print(f"  QISKIT_IBM_TOKEN: {'Set' if os.getenv('QISKIT_IBM_TOKEN') else 'Not set'}")
print(f"  ANTHROPIC_API_KEY: {'Set' if os.getenv('ANTHROPIC_API_KEY') else 'Not set'}")

In [ ]:
# System prompts
COORDINATOR_PROMPT = """You are the Quantum Volume Optimizer coordinating specialized agents.

Your team:
1. backend-analyst: IBM Quantum backend expert (list backends, get properties)
2. qubit-chain-optimizer: Uses algorithmic tools (find_optimal_qv_qubits_tool) to find best qubits
3. transpiler-benchmarker: Circuit optimization expert (compare local vs AI transpilation)
4. qv-experiment-runner: Runs QV experiments on hardware (submit jobs, get results)

Note: QV circuit generation with heavy outputs, HOP calculation, and statistical analysis
are performed locally using helper functions in this notebook.

Produce a report with: Executive Summary, Backend Analysis, Optimal Qubit Subsets,
Transpilation Comparison, and QV Recommendation."""

BACKEND_ANALYST_PROMPT = """You are the Backend Analyst. List backends, get properties,
find suitable systems for QV experiments."""

QUBIT_CHAIN_PROMPT = """You are the Qubit Chain Optimizer with access to algorithmic tools:
- find_optimal_qv_qubits_tool: Find densely connected subgraphs for QV (use this!)
- find_optimal_qubit_chains_tool: Find optimal linear chains
- get_coupling_map_tool: Get backend connectivity

For QV experiments, use find_optimal_qv_qubits_tool with metric='qv_optimized'."""

TRANSPILER_PROMPT = """You are the Transpiler Benchmarker. Compare local vs AI-powered
optimization, minimize circuit depth and two-qubit gates."""

QV_EXPERIMENT_RUNNER_PROMPT = """You are the QV Experiment Runner, expert in executing QV experiments.

Your role:
1. Run QV circuits on real quantum hardware
2. Monitor job completion
3. Report measurement results for HOP calculation

Available MCP tools:
- run_sampler_tool: Submit circuit to hardware
- get_job_status_tool: Check if job is complete (poll until DONE)
- get_job_results_tool: Get measurement counts from completed job

Note: QV circuit generation, HOP calculation, and statistical analysis are performed
locally using helper functions, not via MCP tools."""

In [ ]:
from typing import Any


def get_mcp_config():
    return {
        "qiskit-ibm-runtime": {
            "transport": "stdio",
            "command": "qiskit-ibm-runtime-mcp-server",
            "args": [],
            "env": {
                "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                "QISKIT_IBM_RUNTIME_MCP_INSTANCE": os.getenv("QISKIT_IBM_RUNTIME_MCP_INSTANCE", ""),
            },
        },
        "qiskit": {
            "transport": "stdio",
            "command": "qiskit-mcp-server",
            "args": [],
            "env": {},
        },
        "qiskit-ibm-transpiler": {
            "transport": "stdio",
            "command": "qiskit-ibm-transpiler-mcp-server",
            "args": [],
            "env": {"QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", "")},
        },
    }


def generate_qv_qasm(num_qubits: int, depth: int | None = None, seed: int = 42) -> str:
    """Generate a true Quantum Volume circuit using Qiskit's library."""
    from qiskit.circuit.library import quantum_volume
    from qiskit.qasm3 import dumps

    qv_circuit = quantum_volume(num_qubits, depth=depth, seed=seed)
    return dumps(qv_circuit.decompose())


def generate_qv_circuit_with_ideal_distribution(
    num_qubits: int,
    depth: int | None = None,
    seed: int | None = None,
) -> dict[str, Any]:
    """Generate a QV circuit and compute its ideal heavy output bitstrings."""
    import logging

    import numpy as np
    from qiskit import QuantumCircuit
    from qiskit.circuit.library import quantum_volume
    from qiskit.qasm3 import dumps
    from qiskit.quantum_info import Statevector

    logger = logging.getLogger(__name__)

    try:
        if num_qubits < 2:
            num_qubits = 2
        elif num_qubits > 10:
            logger.warning(f"QV with {num_qubits} qubits will be slow to simulate.")

        if depth is None:
            depth = num_qubits
        elif depth < 1:
            depth = 1
        elif depth > num_qubits:
            depth = num_qubits

        if seed is None:
            seed = np.random.randint(0, 2**31)

        qv_circuit = quantum_volume(num_qubits, depth=depth, seed=seed)
        qv_decomposed = qv_circuit.decompose()

        statevector = Statevector.from_label("0" * num_qubits)
        final_state = statevector.evolve(qv_decomposed)
        probabilities = final_state.probabilities()

        ideal_probs = {}
        for i, prob in enumerate(probabilities):
            bitstring = format(i, f"0{num_qubits}b")[::-1]
            ideal_probs[bitstring] = prob

        median_prob = float(np.median(probabilities))
        heavy_outputs = [bs for bs, prob in ideal_probs.items() if prob > median_prob]

        qv_with_meas = QuantumCircuit(num_qubits, num_qubits)
        qv_with_meas.compose(qv_decomposed, inplace=True)
        qv_with_meas.measure(range(num_qubits), range(num_qubits))
        qasm3_circuit = dumps(qv_with_meas)

        result = {
            "status": "success",
            "circuit_qasm": qasm3_circuit,
            "num_qubits": num_qubits,
            "depth": depth,
            "seed": seed,
            "heavy_outputs": heavy_outputs,
            "num_heavy_outputs": len(heavy_outputs),
            "median_probability": median_prob,
            "message": f"Generated QV-{num_qubits} circuit with {len(heavy_outputs)} heavy outputs",
        }

        if num_qubits <= 6:
            result["ideal_probabilities"] = ideal_probs

        return result

    except Exception as e:
        logger.error(f"Failed to generate QV circuit: {e}")
        return {"status": "error", "message": f"Failed to generate QV circuit: {e!s}"}


def calculate_heavy_output_probability(
    counts: dict[str, int],
    heavy_outputs: list[str],
) -> dict[str, Any]:
    """Calculate the Heavy Output Probability (HOP) for QV validation."""
    import logging

    logger = logging.getLogger(__name__)

    try:
        if not counts:
            return {"status": "error", "message": "No counts provided"}

        if not heavy_outputs:
            return {"status": "error", "message": "No heavy outputs provided"}

        heavy_set = set(heavy_outputs)
        total_shots = sum(counts.values())
        heavy_counts = sum(count for bitstring, count in counts.items() if bitstring in heavy_set)

        hop = heavy_counts / total_shots if total_shots > 0 else 0.0
        threshold = 2 / 3
        above_threshold = hop > threshold

        return {
            "status": "success",
            "heavy_output_probability": hop,
            "total_shots": total_shots,
            "heavy_counts": heavy_counts,
            "num_heavy_bitstrings": len(heavy_outputs),
            "threshold": threshold,
            "above_threshold": above_threshold,
            "message": f"HOP = {hop:.4f} ({'above' if above_threshold else 'below'} threshold)",
        }

    except Exception as e:
        logger.error(f"Failed to calculate HOP: {e}")
        return {"status": "error", "message": f"Failed to calculate HOP: {e!s}"}


def analyze_qv_experiment_results(
    hop_values: list[float],
    confidence_level: float = 0.975,
) -> dict[str, Any]:
    """Analyze results from multiple QV circuit runs."""
    import logging

    import numpy as np
    from scipy import stats

    logger = logging.getLogger(__name__)

    try:
        if not hop_values:
            return {"status": "error", "message": "No HOP values provided"}

        hop_array = np.array(hop_values)
        n = len(hop_array)

        if n < 10:
            logger.warning(f"Only {n} HOP values. Recommend at least 100.")

        mean_hop = float(np.mean(hop_array))
        std_hop = float(np.std(hop_array, ddof=1))
        sem = std_hop / np.sqrt(n)

        t_critical = stats.t.ppf(confidence_level, df=n - 1)
        ci_lower = mean_hop - t_critical * sem
        ci_upper = mean_hop + t_critical * sem

        threshold = 2 / 3
        qv_achieved = bool(ci_lower > threshold)
        margin = float(ci_lower - threshold)

        message = (
            f"QV {'ACHIEVED' if qv_achieved else 'NOT achieved'}! "
            f"Mean HOP = {mean_hop:.4f}, CI lower = {ci_lower:.4f}"
        )

        return {
            "status": "success",
            "qv_achieved": qv_achieved,
            "mean_hop": mean_hop,
            "std_hop": std_hop,
            "standard_error": sem,
            "confidence_interval": (ci_lower, ci_upper),
            "confidence_level": confidence_level,
            "num_circuits": n,
            "threshold": threshold,
            "margin": margin,
            "message": message,
        }

    except Exception as e:
        logger.error(f"Failed to analyze QV results: {e}")
        return {"status": "error", "message": f"Failed to analyze: {e!s}"}

In [ ]:
async def create_agent():
    mcp_config = get_mcp_config()
    mcp_client = MultiServerMCPClient(mcp_config)

    # Load tools using get_tools() which creates self-managing tools
    # that handle their own sessions (new session per tool call)
    all_tools, server_tools = [], {}
    for name in mcp_config:
        try:
            tools = await mcp_client.get_tools(server_name=name)
            server_tools[name] = tools
            all_tools.extend(tools)
            print(f"{name}: {len(tools)} tools")
        except Exception as e:
            print(f"{name}: FAILED - {e}")

    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0, max_tokens=8192)

    subagents = [
        {
            "name": "backend-analyst",
            "description": "IBM Quantum backend expert",
            "system_prompt": BACKEND_ANALYST_PROMPT,
            "tools": server_tools.get("qiskit-ibm-runtime", []),
        },
        {
            "name": "qubit-chain-optimizer",
            "description": "Topology analysis expert with algorithmic qubit finding tools",
            "system_prompt": QUBIT_CHAIN_PROMPT,
            "tools": server_tools.get("qiskit-ibm-runtime", []),  # Has QV qubit tools
        },
        {
            "name": "transpiler-benchmarker",
            "description": "Circuit optimization expert",
            "system_prompt": TRANSPILER_PROMPT,
            "tools": server_tools.get("qiskit", []) + server_tools.get("qiskit-ibm-transpiler", []),
        },
        {
            "name": "qv-experiment-runner",
            "description": "Expert in running QV experiments on hardware (submit jobs, get results)",
            "system_prompt": QV_EXPERIMENT_RUNNER_PROMPT,
            "tools": server_tools.get("qiskit-ibm-runtime", []),
        },
    ]

    return create_deep_agent(
        model=llm, tools=all_tools, system_prompt=COORDINATOR_PROMPT, subagents=subagents
    )


agent = await create_agent()
print("\nAgent ready!")

In [ ]:
# Run optimization
request = f"""
Find the optimal Quantum Volume configuration for my IBM Quantum account.

1. Discover available backends (use backend-analyst)
2. Use find_optimal_qv_qubits_tool to find optimal qubit subsets for QV depths 2-5
3. Compare local vs AI transpilation for the best qubit configurations
4. Generate recommendation report with specific qubits and expected QV

Sample QV-4 circuit for transpilation testing:
```qasm
{generate_qv_qasm(4)}
```
"""

result = await agent.ainvoke({"messages": [{"role": "user", "content": request}]})
print(result.get("messages", [])[-1].content if result.get("messages") else "No response")

In [ ]:
# Interactive follow-up
while True:
    query = input("You: ").strip()
    if query.lower() in ["quit", "exit", "q"]:
        break
    if not query:
        continue
    result = await agent.ainvoke({"messages": [{"role": "user", "content": query}]})
    print(
        f"\nAssistant: {result.get('messages', [])[-1].content if result.get('messages') else 'No response'}\n"
    )